<a href="https://colab.research.google.com/github/mancher07/EmiReproductor/blob/main/Java_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Give me a name {display-mode: "form"}

# This code will be hidden when the notebook is loaded.


from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import ipywidgets as widgets
from IPython.display import display, HTML

# Cargar el modelo y el tokenizador
checkpoint = "Salesforce/codegen-350M-mono"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Función para generar respuestas del modelo con manejo de errores
def generate_response(prompt):
    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        if inputs['input_ids'].numel() == 0:
            return "Error: No input provided."
        max_length = 1500  # Aumentar la longitud máxima para generación
        completion = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs.get('attention_mask'),
            max_length=max_length,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        return tokenizer.decode(completion[0], skip_special_tokens=True)
    except Exception as e:
        return f"Error: {str(e)}"

# Crear campos de entrada y botones
input_box = widgets.Textarea(
    description='',
    placeholder='Type your message here...',
    layout=widgets.Layout(width='100%', height='100px', margin='0px 0px 10px 0px')
)
send_button = widgets.Button(description="Send")

# Crear un área de salida desplazable y con mayor capacidad
output_box = widgets.HTML(
    value='',
    placeholder='Bot replies will appear here...',
    layout=widgets.Layout(width='100%', height='400px', overflow_y='auto', border='solid 1px gray')
)

# Contenedor para toda la aplicación
container = widgets.VBox([output_box, input_box, send_button])

# CSS para personalizar el estilo de los mensajes, los avatares y el ícono de verificado
css = """
<style>
    .message-container {
        display: flex;
        align-items: flex-start;
        margin: 10px 0;
        max-width: 100%;  /* Ajustar al 100% del contenedor padre */
        word-wrap: break-word;
    }
    .message-avatar {
        width: 40px;
        height: 40px;
        border-radius: 50%;
        margin: 0 10px;  /* Asegura un espacio entre la imagen y la burbuja */
    }
    .message-box {
        padding: 10px;
        border-radius: 15px;  /* Borde redondeado */
        max-width: 80%;  /* Limita el ancho de la burbuja para que no ocupe todo el espacio */
        display: flex;
        align-items: center;
        flex-grow: 1;  /* Permite que la burbuja crezca y se ajuste al contenido */
        box-sizing: border-box;  /* Incluye el padding en el ancho y alto */
    }
    .user-message {
        background-color: #e7e7e7;
        flex-direction: row-reverse;
        text-align: left;
    }
    .bot-message {
        background-color: #d1ffd1;
        flex-direction: row;
        text-align: left;
    }
    .message-name {
        font-weight: bold;
        margin-bottom: 5px;
        display: flex;
        align-items: center;
    }
    .message-name img {
        margin-left: 5px;
    }
    .verified-icon {
        width: 16px;
        height: 16px;
    }
</style>
"""

def on_send_button_click(b):
    user_message = input_box.value
    if user_message.strip():  # Verificar que el mensaje no esté vacío
        bot_reply = generate_response(user_message)
        # Mostrar el mensaje del usuario y la respuesta del bot en el área de salida
        output_box.value += f"""
        <div class="message-container user-message">
            <img src="https://static.wikia.nocookie.net/yugiohenespanol/images/8/8b/Foto_invitaci%C3%B3n_avatar_doble.jpg/revision/latest?cb=20200827063040&path-prefix=es" alt="User Avatar" class="message-avatar">
            <div>
                <div class="message-name">You <img src="https://iconape.com/wp-content/png_logo_vector/google-verified.png" alt="Verified" class="verified-icon"></div>
                <div class="message-box">
                    {user_message}
                </div>
            </div>
        </div>
        <div class="message-container bot-message">
            <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR0m46PamPDvagmS__4zoTLlAAhUq7vL0fTbQ&s" alt="Bot Avatar" class="message-avatar">
            <div>
                <div class="message-name">Bot</div>
                <div class="message-box">
                    {bot_reply}
                </div>
            </div>
        </div>
        """
        # Limpiar el campo de entrada después del envío
        input_box.value = ''
    else:
        output_box.value += """
        <div class="message-container user-message">
            <img src="https://static.wikia.nocookie.net/yugiohenespanol/images/8/8b/Foto_invitaci%C3%B3n_avatar_doble.jpg/revision/latest?cb=20200827063040&path-prefix=es" alt="User Avatar" class="message-avatar">
            <div>
                <div class="message-name">You <img src="https://iconape.com/wp-content/png_logo_vector/google-verified.png" alt="Verified" class="verified-icon"></div>
                <div class="message-box">
                    (empty message)
                </div>
            </div>
        </div>
        <div class="message-container bot-message">
            <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR0m46PamPDvagmS__4zoTLlAAhUq7vL0fTbQ&s" alt="Bot Avatar" class="message-avatar">
            <div>
                <div class="message-name">Bot</div>
                <div class="message-box">
                    Please enter a message.
                </div>
            </div>
        </div>
        """

send_button.on_click(on_send_button_click)

# Mostrar los widgets en la interfaz de usuario
display(HTML(css))  # Insertar el CSS para el estilo
display(container)